<h3>FP2 Dataset</h3>

#### Two sources of data for each company:
<ol>
    <li>Text fom their financial reports</li>
    <li>Time series of their stock price, volume, etc</li>
</ol>

### Prices

Base URL: https://api.koyfin.com/api/v2/commands/g/g.eq/

AMZN?dateFrom=1900-10-15&dateTo=2020-10-30&period=daily

ticker + "?dateFrom=" + Year-month-day + "&dateTo=" + Year-month-day + "&period=daily"

In [13]:
import requests

# Return json dictionary of pricing information given ticker
def request_json(ticker: str):
    url_base = "https://api.koyfin.com/api/v2/commands/g/g.eq/"
    default_date = "?dateFrom=1900-10-15&dateTo=2020-10-30&period=weekly"
    r = requests.get(url_base + ticker + default_date)
    print("\nRequest Response {}\n".format(r.status_code))
    return r.json()

In [17]:
# example request
json_data = request_json("Z")


Request Response 200



In [18]:
import pandas as pd

# given ticker constructs dataframe of available pricing info
def json_df(ticker: str):
    json_data = request_json(ticker)
    json_prices = json_data["graph"]["data"]
    truncated_prices = [date[0:6] for date in json_prices]

    column_label = ["Date", "Open", "High", "Low", "Close/Last", "Volume"]

    df = pd.DataFrame(truncated_prices, columns=column_label) 
    df.insert(0,"Ticker","")
    df["Ticker"] = json_data["ticker"]
    return df

In [19]:
json_df("Z")


Request Response 200



,Ticker,Date,Open,High,Low,Close/Last,Volume
0,Z,2015-08-07,27.65,28.82,22.99,26.50,507323
1,Z,2015-08-14,25.76,26.60,24.56,25.19,697870
2,Z,2015-08-21,26.58,27.09,24.35,25.18,11811515
3,Z,2015-08-28,23.93,26.25,23.00,24.86,9995228
4,Z,2015-09-04,24.66,26.20,24.01,25.07,5397081
...,...,...,...,...,...,...,...
269,Z,2020-10-02,102.85,109.67,100.07,108.18,13711191
270,Z,2020-10-09,109.81,112.49,103.52,104.26,13451992
271,Z,2020-10-16,106.48,106.48,97.08,97.11,14742773
272,Z,2020-10-23,98.54,101.85,94.65,96.16,11019821


In [20]:
json_df("AAPL")


Request Response 200



,Ticker,Date,Open,High,Low,Close/Last,Volume
0,AAPL,1982-10-08,0.083705,0.105468,0.080357,0.104910,910313600
1,AAPL,1982-10-15,0.107142,0.110491,0.101004,0.102678,1094800000
2,AAPL,1982-10-22,0.104910,0.119419,0.102678,0.115513,848489600
3,AAPL,1982-10-29,0.108816,0.116071,0.103794,0.113281,879648000
4,AAPL,1982-11-05,0.119419,0.142299,0.112165,0.134486,1120918400
...,...,...,...,...,...,...,...
1982,AAPL,2020-10-02,115.010000,117.720000,112.220000,113.020000,641240500
1983,AAPL,2020-10-09,113.910000,117.000000,112.250000,116.970000,548575040
1984,AAPL,2020-10-16,120.060000,125.390000,118.150000,119.020000,881572600
1985,AAPL,2020-10-23,119.960000,120.419000,114.280000,115.040000,519569630


In [21]:
json_df("AMZN").to_csv("amzn_info.csv")


Request Response 200



### Text of Reports